In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict, defaultdict
import os
import pyfpgrowth;
from efficient_apriori import apriori

datapath = os.getcwd()
log_data_path = datapath + "/data" #all log data should kept in this folder
result_save_path = datapath + "/result" # all generated txt result save here
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [27]:
chrome_log = log_data_path + "/Log.txt.strace"
data = pd.read_csv(chrome_log , sep=" ",header=None,names=["Name","Junk1","Junk2","DateTime","Permission","FilePath","Received","BytesAccessed"])

In [28]:
type(data)

pandas.core.frame.DataFrame

In [29]:
data=data.drop(['Junk1','Junk2'], axis=1)

In [30]:
data['DateTimePermission'] = data['DateTime'].str.cat(data['Permission'])

In [31]:
data=data.drop(['DateTime','Permission'], axis=1)

In [32]:
data['FilePath']=data['FilePath'].str.replace('GET ','')

In [33]:
data['DateTimePermission']=data['DateTimePermission'].str.replace('-0700','')

In [34]:
data=data.rename(index=str,columns={"DateTimePermission":"DateTime"})

In [35]:
'''data= pd.DataFrame(['24/Jul/2018:15:39:48'],columns=['DateTime'])
data['DateTime'] = data['DateTime'].apply(lambda x:pd.datetime.strptime(x,'%d/%b/%Y:%H:%M:%S'))'''

"data= pd.DataFrame(['24/Jul/2018:15:39:48'],columns=['DateTime'])\ndata['DateTime'] = data['DateTime'].apply(lambda x:pd.datetime.strptime(x,'%d/%b/%Y:%H:%M:%S'))"

In [36]:
#Preprocessing ::#Dropped the Name and Received column
data=data.drop(['Name','Received'],axis=1)
print (data.FilePath.size)

687405


In [68]:
"""
Pass the DF before any preprocessing to generate reply log and uncluster baseline txt
"""
def createReplyLog_And_BaseLine(dataFrame):
    replay_Log_Name = os.path.join(result_save_path, "replay_log_user.txt")
    freplay = open(replay_Log_Name,"w+")  
    for file in dataFrame.FilePath:
        freplay.write(file+"\n")
    freplay.close()

    #Write the cluster.index_base :: all unique filePath as unclustered: As baseline to compare our results
    baseCluster_File_Name = os.path.join(result_save_path, "cluster.index_baseline_user.txt")    
    fbaseCluster = open(baseCluster_File_Name,"w+")
    fbaseCluster.write("Unclustered Files:\n")
    for file in data.FilePath.unique():
        if file=="":
            continue
        fbaseCluster.write(file+"\n")

    fbaseCluster.close()

In [69]:
#reply Log and baseline cluster need to be created only Once, If u need uncomment below and run
#createReplyLog_And_BaseLine(data)

In [37]:
#Preprocessing :: Remove files those does not have any . in its file path name
print ("Before dot preprocessing data size: ", data.FilePath.size)
data_filtered = (data.set_index('FilePath').filter(like='.', axis=0).reset_index())[data.columns]
print ("After dot preprocessing data size: ",data_filtered.FilePath.size)

#Rechekcing - number of valid files
# count = 0
# for i in data.FilePath:
#     if '.' in i:
#         count = count + 1
# print (count)


Before dot preprocessing data size:  687405
After dot preprocessing data size:  28359


In [40]:
#Set a time window to prepare transactions:: timed_window no of different timed accessed files will be on same transaction.

#Number_of_transactions
#All secone based transaction will be in this list
time_transactions = []
time_transactions.clear()

#temp-> to be pushed into time_transactions
#tempTransactions = []
tempTransactions = set()
tempTransactions.clear()
print (type(tempTransactions))

previous_date_time = data_filtered.DateTime[0] # just to avoid error
print ("Total Data Size: ", data_filtered.FilePath.size)  

#sample data 
#print (data_filtered.FilePath[0])

timed_window = 4;  # Time Window -- Can set the number of different time window we want in one transaction
count_window = 0; #ref counting

for i in range (0,data_filtered.FilePath.size-1):
    try:

        #if time-stamp is different then push the temporary list into the actual transaction
        if(previous_date_time != data_filtered.DateTime[i]):
            count_window = count_window + 1
            #print ("Debug Log:",count_window, " ", data_filtered.DateTime[i]) 
            if(count_window>=timed_window):
             #   print ("\n\n")
                count_window = 0
              #  print ("Debug Log:",count_window, " ", data_filtered.DateTime[i]) 
                tempTransactionsList = list(tempTransactions)
                tempTransactionsList = [x.strip(' ') for x in tempTransactionsList]
                time_transactions.append(tempTransactionsList)
                # Need to convert before pushed into transaction
                tempTransactions.clear()  
                
            
            
        tempTransactions.add(data_filtered.FilePath[i])                           
        previous_date_time = data_filtered.DateTime[i] #update previous timestamp
        
    except Exception as e:
        print("Exception: ", e)


print ("No of Time Based Transactions: \n", len(time_transactions))

#print (time_transactions)



<class 'set'>
Total Data Size:  28359
No of Time Based Transactions: 
 126


In [56]:
"""
It generate 2 txt file: 
    1. Cluster List in txt with your given name of file
       cluster.index fileName Convention: cluster.index_algorithm_supportThreshold_timeWindow
       
    2. All filePath list from dataFrame with yoru given name of file
    
Args:
        patterns_keyList: List of list (each inner list presents a cluster)
        DataFrame_filepath: Data Frame file path list
        Algo_Name: Algorithm to generate this frequent pattern        
        supportThreshold :: used in the algorithm
        TimeWindow: which has been used in our transaction preprocessing
Optional Arguments: 
        Feature :: features consider to create transaction, ex : time only, time+directory
"""
def createClusterFile(patterns_keyList, DataFrame_filepath, algoName, supportThreshold, TimeWindow, feature =""):

    #---------------Results file path-----------------#    
    #append the result directory
    #DataPath_Name = os.path.join(result_save_path, DataPath_Name+".txt")

    cluster_File_Name = "cluster.index_" + algoName + "_Support=" + str(supportThreshold) + "_timeWindow="+ str(TimeWindow) + "_Features="+feature
    #append the result directory
    cluster_File_Name = os.path.join(result_save_path, cluster_File_Name+".txt")              
    print ("Cluster File Name: ", cluster_File_Name)
    #--------------------------------------------------#
    fCluster = open(cluster_File_Name,"w+")    
    count = 1;
    hash_Dict = {}
    for items in patterns_keyList:
        inner_keyList = list(items)
        if(len(inner_keyList)==1): ## remove single file cluster
            continue
        fCluster.write("Cluster %d\n" % (count))
        count = count + 1 
        for i in inner_keyList:
            #print(i)
            fCluster.write("%s\n" %i)
            if i not in hash_Dict:
                hash_Dict[i] = True    

    #create the unclustered filepath
    fCluster.write("Unclustered Files:\n")
    #filePath = open(DataPath_Name, "w+")

    #------------Uncluster Files in cluster.index and all FilePath in DataPath txt-------------#
    for i in DataFrame_filepath:
      #  filePath.write("%s\n"%(i.strip(' '))) #all data frame files
        if i not in hash_Dict:
            fCluster.write("%s\n"%i) #uncluster files
            hash_Dict[i] = True #Removing duplicates in uncluster list

    fCluster.close()  
    #filePath.close()

In [42]:
#---------------closed Itemset calculation from frequent itemset------------
def cal_closed_itemset(itemsetList):

    maximal_itemset = []
    for index_i in range (0,len(itemsetList)): 
        cluster_i_set = set(itemsetList[index_i])    
        flag = True
        for index_j in range (index_i+1, len(itemsetList)):
            cluster_j_set = set(itemsetList[index_j])    
            if(cluster_i_set.issubset(cluster_j_set)):
                #print (cluster_i_set , "<=>", cluster_j_set)
                flag = False
                break
        if(flag):
            maximal_itemset.append(itemsetList[index_i])
    #print ("Maximal Itemset:", maximal_itemset)
    print ("size of maximal Itemsets", len(maximal_itemset))
    return maximal_itemset

In [59]:
## Try FP_Growth algorithm on this transaction set
#print (time_transactions)

#Use find_frequent_patterns to find patterns in baskets that occur over the support threshold:

minSupport = 20 #experimental feature
patterns = pyfpgrowth.find_frequent_patterns(time_transactions, minSupport)  # need to generate data using this param
print ("FP Growth pattern:")
# for key,val in patterns.items():
#    print (key, "=>", val)
itemsetList = []
for key,val in patterns.items():
#    print (key ,"\n")
    itemsetList.append(key)
    
#print ("All frequent items: ", itemsetList)
print ("Size of All frequest Itemset: ", len(itemsetList))
closed_itemset = cal_closed_itemset(itemsetList)

feature = "time"
createClusterFile(closed_itemset, data_filtered.FilePath, "FPGrowth",minSupport,timed_window,feature)
    
#Use generate_association_rules to find patterns that are associated with another with a certain minimum probability:
# minProbability = 0.5
# rules = pyfpgrowth.generate_association_rules(patterns, minProbability)  # generate data using this prob 

# print("\nFP Growth Association Rules :")
# for key,val in rules.items():
#     print (list(key) , "=>" , list(val))

FP Growth pattern:
Size of All frequest Itemset:  12
size of maximal Itemsets 8


In [60]:
##Try Apriori Algorithm with features minSup, minCon,
minSupport = 0.10
itemsets, rules = apriori(time_transactions, minSupport)
#print ("Apriori pattern:", itemsets)

#Formating the itemsets 
itemsetList = []
for key,val in itemsets.items():
    temp = ()
    for i in val:
        temp+=(i) #tuple concatenation
    itemsetList.append(tuple(set(temp)))  # remove duplicate from each cluster
print ("\n\nApriori List: ", len(itemsetList))
#--------------------------------//  

#print ("All frequent items: ", itemsetList)
print ("Size of All frequest Itemset: ", len(itemsetList))
closed_itemset = cal_closed_itemset(itemsetList)


feature = "time"
createClusterFile(closed_itemset, data_filtered.FilePath,"Apriori_cluster",minSupport, timed_window,feature)

    




Apriori List:  4
Size of All frequest Itemset:  4
size of maximal Itemsets 2


In [45]:
FrequencyDict=data.FilePath.value_counts()

In [46]:
FrequencyDict=FrequencyDict.to_dict()

In [47]:
type(FrequencyDict)

dict

In [48]:
def convertFrequencyDict(columns,FrequencyDict):
    frequency=str(columns[0])
    try:
        Frequency=FrequencyDict[frequency]
    except Exception as e:
        Frequency=0
    return Frequency

In [49]:
data['Frequency']=data[['FilePath']].apply(convertFrequencyDict,args=(FrequencyDict,),axis=1)

In [50]:
data

,FilePath,BytesAccessed,DateTime,Frequency
0,/lib/x86_64-linux-gnu/libutil.so.1,832.0,[28/Jun/2018:11:53:09],2
1,/usr/lib/x86_64-linux-gnu/libutempter.so.0,832.0,[28/Jun/2018:11:53:09],1
2,/lib/x86_64-linux-gnu/libtinfo.so.5,832.0,[28/Jun/2018:11:53:09],4
3,/usr/lib/x86_64-linux-gnu/libevent-2.1.so.6,832.0,[28/Jun/2018:11:53:09],1
4,/lib/x86_64-linux-gnu/libresolv.so.2,832.0,[28/Jun/2018:11:53:09],7
5,/lib/x86_64-linux-gnu/libc.so.6,832.0,[28/Jun/2018:11:53:09],51
6,/lib/x86_64-linux-gnu/libpthread.so.0,832.0,[28/Jun/2018:11:53:09],32
7,/etc/localtime,1918.0,[28/Jun/2018:11:53:09],18
8,/etc/localtime,1226.0,[28/Jun/2018:11:53:09],18
9,/etc/localtime,1.0,[28/Jun/2018:11:53:09],18
